In [ ]:
# Example of various pivpy methods
import sys
import pathlib
import importlib.util

print(f'Python executable: {sys.executable}')
print(f'Working directory: {pathlib.Path.cwd().resolve()}')

spec = importlib.util.find_spec('pivpy')
repo_root = None

if spec is None:
    # 1) Try importing directly from the repo checkout by adding it to sys.path (works immediately).
    cwd = pathlib.Path.cwd().resolve()
    for p in [cwd] + list(cwd.parents):
        if (p / 'pyproject.toml').exists() and (p / 'pivpy').is_dir():
            repo_root = p
            break
    if repo_root is not None:
        sys.path.insert(0, str(repo_root))
        print(f'Added repo root to sys.path: {repo_root}')
        spec = importlib.util.find_spec('pivpy')

print('pivpy importable:', spec is not None)
if spec is None:
    raise RuntimeError('pivpy is not importable in this kernel. Select the pivpy (.venv) kernel or open this notebook from inside the repository.')
import pivpy
print('pivpy version:', getattr(pivpy, '__version__', 'unknown'))
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print("\nChecking installed packages:")

pivpy_spec = importlib.util.find_spec('pivpy')
if pivpy_spec is None:
    print('pivpy: NOT INSTALLED in this kernel')
else:
    import pivpy
    print(f"pivpy version: {getattr(pivpy, '__version__', 'installed (version unavailable)')}")

try:
    import xarray as xr
    print(f"xarray version: {xr.__version__}")
except ImportError:
    print("xarray: NOT INSTALLED")

In [ ]:
import importlib.util
if importlib.util.find_spec('pivpy') is None:
    raise RuntimeError("pivpy is not available in this kernel. Run the install cell above or switch the notebook kernel to pivpy (.venv).")

from pivpy import io, pivpy, graphics
import numpy as np
import xarray as xr
import importlib.resources
import pathlib

import matplotlib.pyplot as plt

In [ ]:
f1 = 'Run000001.T000.D000.P000.H001.L.vec'
f2 = 'Run000002.T000.D000.P000.H001.L.vec'

# Ensure compatibility with different Python versions (3.9+ has 'files', 3.7 and 3.8 need 'path')
try:
    from importlib.resources import files
except ImportError:
    from importlib.resources import path as resource_path

# For Python 3.9+
try:
    path = files('pivpy') / 'data'
except NameError:
    # For Python 3.7 and 3.8
    with resource_path('pivpy', 'data') as data_path:
        path = pathlib.Path(data_path)



a = io.load_vec(path / "Insight" / f1 )
b = io.load_vec(path / "Insight" / f2 )

In [ ]:
# select where t = 1 (explicit time)
fig, ax = graphics.quiver(a.sel(t=1),scalingFactor=10)

#increase figure size
fig.set_size_inches(11,8)

In [ ]:
# select just the first frame whatever t is .
b['t'] = b['t'] + 10
# define size before the plot
plt.figure(figsize=(11,8))

# show less vectors using nthArr 
fig, ax = graphics.quiver(b.isel(t=0),scalingFactor=10)

In [ ]:
c = a.piv.crop([5, 15,-5,-15])
a.u.shape, c.u.shape

In [ ]:
# define size before the plot
plt.figure(figsize=(11,8))

# show less vectors using nthArr 
fig, ax = graphics.quiver(c.isel(t=0),scalingFactor=10)

In [ ]:
# let's play with some synthetic data 
c = io.create_sample_Dataset()

In [ ]:
# want to slice it and not crop? 
d = c.sel(x = slice(35,70),y=slice(30,90))
print(d)

In [ ]:
# want to show an ensemble average of 10 frames?
data = io.create_sample_Dataset(10)

# want to change the size of arrows and figure aspectratio?
fig,_=graphics.quiver(data.piv.average,scalingFactor=80)
fig.set_size_inches(11,8)


In [ ]:
data = io.create_sample_field()
data.piv.strain()


In [ ]:
data.piv.vorticity()
data